# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score,make_scorer
from sklearn.neighbors import KNeighborsClassifier
import pickle
import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Johnny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Johnny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///notebooks/processeddata.db')

df = pd.read_sql("SELECT * FROM df", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z]", " ", text.lower())
    tokens = word_tokenize(text)
    stemmed = [PorterStemmer().stem(word) for word in tokens if word not in stopwords.words("english")]

    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(verbose=False)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(233)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def get_score(y_true, y_pred, target_names):
    df = pd.DataFrame()
    for i,target in enumerate(target_names):
        accuracy = accuracy_score(y_true[:, i], y_pred[:, i])
        f1 = f1_score(y_true[:, i], y_pred[:, i])
        precision = precision_score(y_true[:, i], y_pred[:, i])
        recall = recall_score(y_true[:, i], y_pred[:, i])
        df = df.append({'index':target,'Accuracy':accuracy,'F1 Score':f1,'Precision':precision,'Recall':recall},ignore_index = True)
    return df

In [9]:
target_names = Y.columns

In [10]:
y_train_pred = pipeline.predict(X_train)
y_train_true = np.array(Y_train)

In [11]:
result_train_df = get_score(y_train_true, y_train_pred, target_names)
display(result_train_df)

,Accuracy,F1 Score,Precision,Recall,index
0,0.990625,0.993881,0.992918,0.994846,related
1,0.987245,0.961592,0.996165,0.929338,request
2,0.998924,0.875740,1.000000,0.778947,offer
3,0.985554,0.982567,0.993996,0.971397,aid_related
4,0.987654,0.916580,1.000000,0.846006,medical_help
5,0.992470,0.920152,0.997644,0.853831,medical_products
6,0.993494,0.875123,0.997758,0.779335,search_and_rescue
7,0.995594,0.858553,1.000000,0.752161,security
8,0.996158,0.937448,1.000000,0.882261,military
9,0.996312,0.970492,1.000000,0.942675,water


In [12]:
y_true = np.array(Y_test)
y_pred = pipeline.predict(X_test)

In [13]:
result_pred_df = get_score(y_true, y_pred, target_names)
display(result_pred_df)

,Accuracy,F1 Score,Precision,Recall,index
0,0.803750,0.876152,0.845247,0.909402,related
1,0.885969,0.573563,0.804839,0.445536,request
2,0.996465,0.000000,0.000000,0.000000,offer
3,0.746427,0.660773,0.735469,0.599851,aid_related
4,0.923313,0.191248,0.602041,0.113680,medical_help
5,0.953435,0.192000,0.666667,0.112150,medical_products
6,0.977255,0.075000,0.857143,0.039216,search_and_rescue
7,0.980790,0.015748,0.333333,0.008065,security
8,0.967727,0.160000,0.740741,0.089686,military
9,0.958506,0.551495,0.892473,0.399038,water


### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
def mid_f1(y_true, y_pred):
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [20]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[5, 10]}
scorer = make_scorer(mid_f1)
cv = GridSearchCV(pipeline, param_grid = parameters,scoring = scorer,verbose = 10, n_jobs=-1)

In [21]:
np.random.seed(233)
tuned_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 24.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 41.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 49.9min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 75.3min
[Parallel(n_jobs=-1)]: Done  46 out of  48 | elapsed: 100.3min remaining:  4.4min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 101.9min finished


In [24]:
print(tuned_model.best_params_)
print(cv.best_score_)

{'clf__estimator__min_samples_split': 10, 'clf__estimator__n_estimators': 10, 'tfidf__use_idf': True, 'vect__min_df': 5}
0.22485969644939335


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [25]:
y_true = np.array(Y_test)
y_tuned_pred = tuned_model.predict(X_test)
result_tuned_df = get_score(y_true, y_tuned_pred, target_names)
display(result_tuned_df)

,Accuracy,F1 Score,Precision,Recall,index
0,0.812356,0.882945,0.842789,0.927119,related
1,0.893038,0.624595,0.788828,0.516964,request
2,0.996465,0.000000,0.000000,0.000000,offer
3,0.758875,0.705352,0.709751,0.701008,aid_related
4,0.921316,0.235821,0.523179,0.152216,medical_help
5,0.955433,0.275000,0.696203,0.171340,medical_products
6,0.976487,0.072727,0.500000,0.039216,search_and_rescue
7,0.980483,0.030534,0.285714,0.016129,security
8,0.967420,0.258741,0.587302,0.165919,military
9,0.960965,0.612805,0.837500,0.483173,water


In [26]:
display(result_pred_df.describe())
display(result_tuned_df.describe())

,Accuracy,F1 Score,Precision,Recall
count,35.000000,35.000000,35.000000,35.000000
mean,0.942734,0.250453,0.586363,0.193703
std,0.057848,0.270638,0.304362,0.244644
min,0.746427,0.000000,0.000000,0.000000
25%,0.931996,0.027172,0.416667,0.013910
50%,0.958506,0.132450,0.678571,0.072464
75%,0.980636,0.469626,0.810186,0.330582
max,0.996465,0.876152,1.000000,0.909402


,Accuracy,F1 Score,Precision,Recall
count,35.000000,35.000000,35.000000,35.000000
mean,0.945496,0.304772,0.549098,0.247036
std,0.054798,0.285044,0.313969,0.271040
min,0.758875,0.000000,0.000000,0.000000
25%,0.938989,0.031934,0.344350,0.016539
50%,0.959736,0.244186,0.684211,0.152216
75%,0.981789,0.597408,0.788730,0.479065
max,0.996465,0.882945,1.000000,0.927119


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

parameters_kn = {'tfidf__use_idf':[True, False],
                 'clf__estimator__weights':['uniform','distance'],
                 'clf__estimator__leaf_size':[25,35]
                }
cv = GridSearchCV(pipeline, param_grid = parameters_kn,scoring = scorer,verbose = 10, n_jobs=-1)

np.random.seed(233)
kn_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 29.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 48.3min
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed: 48.5min remaining:  9.7min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 59.1min finished


In [28]:
print(kn_model.best_params_)
print(cv.best_score_)

{'clf__estimator__leaf_size': 25, 'clf__estimator__weights': 'distance', 'tfidf__use_idf': True}
0.16819123270736175


In [29]:
y_true = np.array(Y_test)
y_kn_pred = kn_model.predict(X_test)
result_kn_df = get_score(y_true, y_kn_pred, target_names)
display(result_kn_df)
display(result_kn_df.describe())

,Accuracy,F1 Score,Precision,Recall,index
0,0.788996,0.870018,0.821122,0.925106,related
1,0.868910,0.536161,0.685675,0.440179,request
2,0.996312,0.000000,0.000000,0.000000,offer
3,0.693714,0.544040,0.702719,0.443822,aid_related
4,0.921162,0.131980,0.541667,0.075145,medical_help
5,0.953896,0.193548,0.705882,0.112150,medical_products
6,0.976641,0.061728,0.555556,0.032680,search_and_rescue
7,0.980944,0.015873,0.500000,0.008065,security
8,0.965883,0.112000,0.518519,0.062780,military
9,0.946365,0.364299,0.751880,0.240385,water


,Accuracy,F1 Score,Precision,Recall
count,35.000000,35.000000,35.000000,35.000000
mean,0.934598,0.213203,0.531198,0.154322
std,0.068891,0.208447,0.256021,0.190749
min,0.693714,0.000000,0.000000,0.000000
25%,0.919625,0.048014,0.439216,0.025531
50%,0.953896,0.155689,0.625000,0.092199
75%,0.981020,0.318331,0.714052,0.210555
max,0.996312,0.870018,0.857143,0.925106


### 9. Export your model as a pickle file

In [30]:
pickle.dump(tuned_model, open('disaster_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.